# Tarea 3 - Optimización
### Método de Newton con Hessiano Modificado.
#### Por: Gustavo Hernández Angeles


### **Preparación**

#### Funciones de prueba y módulos

In [28]:
import numpy as np
from time import perf_counter

# Definimos las funciones de prueba.
def Esfera(x):
    """
    f(x) = \\sum x_i^2
    """
    
    return np.sum(x**2)


def Rosenbrock(x):
    terminos = [
        100 * (x[i + 1] - x[i] ** 2) ** 2 + (1 - x[i]) ** 2
        for i in range(len(x) - 1)
    ]
    terminos = np.array(terminos)
    return np.sum(terminos)


def Beale(x):
    assert len(x) == 2, f"La función Beale es de dos variables, se introdujeron {len(x)}."
    (x1, x2) = x
    return (
        (1.5 - x1 + x1 * x2) ** 2
        + (2.25 - x1 + x1 * x2**2) ** 2
        + (2.625 - x1 + x1 * x2**3) ** 2
    )

#### Funciones para calcular Hessiano y Gradiente

In [29]:

def hess_f(x, f, h=1e-5):
    """
    Calcula la matriz Hessiana de f en el punto x usando diferencias finitas.
    
    Input:
    f:  función escalar que recibe un vector x
    x:  punto donde se evalúa el Hessiano (numpy array)
    h:  paso pequeño para diferencias finitas

    Output:
    H:  matriz Hessiana (numpy array)
    """
    n = len(x)
    H = np.zeros((n, n))
    I = np.eye(n)  # Matriz identidad para cambiar un elemento a la vez

    for i in range(n):
        for j in range(n):
            x_ij = x + h * (I[i] + I[j])
            x_i = x + h * I[i]
            x_j = x + h * I[j]
            
            f_ij = f(x_ij)  # f(x + h_i + h_j)
            f_i = f(x_i)  # f(x + h_i)
            f_j = f(x_j)  # f(x + h_j)
            f_0 = f(x)  # f(x)

            H[i, j] = (f_ij - f_i - f_j + f_0) / (h**2)  # Segunda derivada cruzada

    return 0.5*(H + H.T)

def grad_f(x, f, h):
    """
    Input:
        f: Función.
        x: Punto a evaluar (numpy array).
        h: Espaciamiento para el cálculo del gradiente.
        
    Output:
        grad: Valor del gradiente (numpy array).
    """
    
    # Inicializa el gradiente
    grad = np.zeros(len(x))

    # Itera sobre cada componente
    for i in range(len(x)):
        # Copia de x
        x_i = np.copy(x)

        # Se suma el espaciamiento solo en la i-esima componente
        x_i[i] = x_i[i] + h
        
        # Se calcula la i-esima componente del gradiente
        grad[i] = (f(x_i) - f(x)) / h
    return grad

#### Condicionar el hessiano

In [30]:
def condicionar_hessiano(hessiano, w = 1e-6, lam = 1e-2, max_iter = 10):
    """
    Asegura que el Hessiano esté bien condicionado
    Input:
        hessiano: Matriz Hessiana (numpy n x n)
        w: Cota menor de min_e/max_e.
    Output:
        Hessiano ajustado (numpy n x n)
    """
    
    H = np.copy(hessiano)
    n = len(H)
    eigenvals = np.linalg.eigvals(H)
    min_e = eigenvals.min()
    max_e = eigenvals.max()

    for i in range(max_iter):
        if min_e / max_e >= w:
            return H
        H += lam*np.eye(n)
        eigenvals = np.linalg.eigvals(H)
        min_e = eigenvals.min()
        max_e = eigenvals.max()
    
    return H

#### Método de Newton con Hessiano Modificado

In [31]:
def newton_hess_modificado(f, x, alfa, max_iter, epsilon, h):
    """
    Input:
        f: Función objetivo.
        x: Punto inicial x_0.
        alfa: Learning rate.
        max_iter: Número máximo de iteraciones.
        epsilon: Criterio de convergencia.
        h: Espaciamiento para el cálculo del gradiente.
        
    Output:
        x like: Punto solución aproximada.
    """
    
    x_k = np.copy(x)
    convergencia = False
    
    for i in range(max_iter):
        # Obtenemos p_k
        hessiano = hess_f(x_k, f, h)
        hessiano = condicionar_hessiano(hessiano)
        grad = grad_f(x_k, f, h)
        p_k = -np.linalg.inv(hessiano).dot(grad)
        
        # Actualizamos la solución
        x_k = x_k + alfa * p_k 
        
        # Evalua la convergencia
        convergencia = max(abs(p_k)) < epsilon
        if convergencia:
            print(f"La función {f.__name__} converge en la iteracion: {i}")
            break

    if not convergencia:
        print(f"No se cumplio la convergencia en {max_iter} iteraciones.")

    return x_k

### **Resultados**

#### Función Esfera

In [32]:
n = 4
x = np.array(n*[4], dtype=float)
alfa = 1
max_iter = 15_000
epsilon = 10e-6
h = 10e-6

inicio = perf_counter()
xsol = newton_hess_modificado(Esfera, x, alfa, max_iter, epsilon, h)
final = perf_counter()

# Comparación con resultado real
solucion = np.array(n*[0], dtype=float)

print(f"Tiempo de ejecución: {(final-inicio) * 10e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(xsol - solucion)}")
print(f"Solución encontrada: {xsol}")

La función Esfera converge en la iteracion: 2
Tiempo de ejecución: 20.25100 ms
Magnitud del vector error: 1.0000000000000003e-05
Solución encontrada: [-5.e-06 -5.e-06 -5.e-06 -5.e-06]


#### Función de Rosenbrock

In [33]:
n = 4
x = np.array(4*[2], dtype=float)
alfa = 1
max_iter = 15_000
epsilon = 10e-6
h = 10e-6

inicio = perf_counter()
xsol = newton_hess_modificado(Rosenbrock, x, alfa, max_iter, epsilon, h)
final = perf_counter()


# Comparación con resultado real
solucion = np.array(n*[1], dtype=float)

print(f"Tiempo de ejecución: {(final-inicio) * 10e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(xsol - solucion)}")
print(f"Solución encontrada: {xsol}")

La función Rosenbrock converge en la iteracion: 10
Tiempo de ejecución: 105.91400 ms
Magnitud del vector error: 0.009179254507642946
Solución encontrada: [0.99899826 0.99800253 0.99601405 0.992039  ]


#### Función de Beale

In [34]:
x = np.array([2,2], dtype=float)
alfa = 1
max_iter = 15_000
epsilon = 10e-6
h = 10e-6

inicio = perf_counter()
xsol = newton_hess_modificado(Beale, x, alfa, max_iter, epsilon, h)
final = perf_counter()



# Comparación con resultado real
solucion = np.array([3, 0.5], dtype=float)

print(f"Tiempo de ejecución: {(final-inicio) * 10e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(xsol - solucion)}")
print(f"Solución encontrada: {xsol}")

La función Beale converge en la iteracion: 7
Tiempo de ejecución: 46.76500 ms
Magnitud del vector error: 3.041381241106071
Solución encontrada: [2.62464136e-08 1.00000001e+00]
